In [14]:
import src.config as config

In [15]:
from datetime import datetime,timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.now()).floor('H')
print(f'{current_date=}')

# we fecth raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

current_date=Timestamp('2023-04-25 11:00:00')


In [16]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date:datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    
    # dowload 2 files from website
    rides = load_raw_data(year=from_date_.year,months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data(year=to_date_.year,months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]
    
    rides = pd.concat([rides,rides_2])
    
    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)
    
    rides.sort_values(by=['pickup_location_id','pickup_datetime'],inplace=True)
    
    return rides

In [17]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

File 2022-03 was already in local storage
File 2022-04 was already in local storage


In [18]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 256/256 [00:00<00:00, 357.84it/s]


In [19]:
import hopsworks

# connecto to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key=['pickup_location_id','pickup_hour'],
    event_time='pickup_hour'
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/30877
Connected. Call `.close()` to terminate connection gracefully.


In [20]:
feature_group.insert(ts_data,write_options={"wait_for_job": False})

Uploading Dataframe: 0.00% |          | Rows 0/172032 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/30877/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x1845f98d430>, None)

In [21]:
ts_data

,pickup_hour,rides,pickup_location_id
0,2023-03-28 11:00:00,3,4
1,2023-03-28 12:00:00,2,4
2,2023-03-28 13:00:00,2,4
3,2023-03-28 14:00:00,2,4
4,2023-03-28 15:00:00,4,4
...,...,...,...
172027,2023-04-25 06:00:00,0,6
172028,2023-04-25 07:00:00,0,6
172029,2023-04-25 08:00:00,0,6
172030,2023-04-25 09:00:00,0,6


In [22]:
ts_data[ts_data['pickup_location_id'] == 221]

,pickup_hour,rides,pickup_location_id
